In [12]:
import pandas as pd
import duckdb as db

In [15]:
df = pd.read_excel("C:/Users/elifs/OneDrive/Masaüstü/watson_healthcare_modified.xlsx")

In [25]:
print(df.columns)

Index(['EmployeeID', 'Age', 'Attrition', 'BusinessTravel', 'DailyRate',
       'Department', 'DistanceFromHome', 'Education', 'EducationField',
       'EmployeeCount', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'Shift',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')


In [64]:
db.query("""
    SELECT Round(AVG(age),2) AS avg_age
    FROM df
""").to_df()

,avg_age
0,36.87


In [55]:
db.query("""
    SELECT  EducationField, Round(AVG(age),2) AS avg_age
    FROM df
    GROUP BY EducationField
""").to_df()

,EducationField,avg_age
0,Medical,36.93
1,Other,35.59
2,Human Resources,37.21
3,Marketing,37.82
4,Technical Degree,36.05
5,Life Sciences,36.88


In [56]:
db.query("""
    SELECT JobRole, Round(AVG(age),2) AS avg_age
    FROM df
    GROUP BY JobRole
""").to_df()

,JobRole,avg_age
0,Admin,46.13
1,Other,35.33
2,Nurse,36.10
3,Therapist,38.58
4,Administrative,45.41


In [68]:
db.query("""
SELECT COUNT(DISTINCT EmployeeID) AS n_age
FROM df
WHERE age > (SELECT AVG(age) FROM df);
""").to_df()

,n_age
0,762
